 # CRF

Import pandas + numpy  and load train + test data

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../datasets/Train_set.csv', encoding='latin1')

# Change de test sets

test = pd.read_csv('../datasets/Test_set.csv')
# test = pd.read_csv('Zero-shot.csv')

Make a list of all words in the train and test set

In [2]:
words = list(set(data['Word'].values))
test_words = list(set(test["Word"].values))

In [3]:
test_n_words = len(test_words)
n_words = len(words)

Sentence getter

In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [5]:
getter = SentenceGetter(data)
test_getter = SentenceGetter(test)

In [6]:
sent = getter.get_next()
test_sent = test_getter.get_next()

In [7]:
sentences = getter.sentences
test_sentences = test_getter.sentences

Get a word's features

In [11]:
def word2features(sent, i):
    word = str(sent[i][0])
    postag = str(sent[i][1])

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = str(sent[i-1][0])
        postag1 = str(sent[i-1][1])
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
        postag1 = str(sent[i+1][1])
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [12]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
test_X = [sent2features(s) for s in test_sentences]
test_y = [sent2labels(s) for s in test_sentences]

Define Train and Test set

In [13]:
X_tr = X
X_te = test_X
y_tr = y
y_te = test_y

In [ ]:
# Code for if the train or test sets need to be shuffled

# train set
# import random
# random.seed(23)
# c = list(zip(X_tr, y_tr))
# random.shuffle(c)
# X_tr, y_tr = zip(*c)

# test set
# import random
# random.seed(23)
# c = list(zip(X_te, y_te))
# random.shuffle(c)
# X_te, y_te = zip(*c)

In [14]:
!pip install sklearn_crfsuite
#!pip install --upgrade pip

The Model

In [15]:
from sklearn_crfsuite import CRF

In [16]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)

In [17]:
crf.fit(X_tr, y_tr)
y_pred = crf.predict(X_te)

In [15]:
import eli5
eli5.show_weights(crf, top=10)

/opt/anaconda3/envs/deep/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/deep/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Evaluation

In [19]:
!pip install nervaluate

In [18]:
from nervaluate import Evaluator
from sklearn_crfsuite import metrics

In [19]:
print(metrics.flat_classification_report(y_te, y_pred))

              precision    recall  f1-score   support

           B       0.91      0.65      0.76       675
           I       0.88      0.72      0.79      1215
           O       0.98      1.00      0.99     30547

    accuracy                           0.98     32437
   macro avg       0.92      0.79      0.85     32437
weighted avg       0.98      0.98      0.98     32437



In [20]:
test_sents_labels = []
for sentence in test_sentences:
    sentence = [token[2] for token in sentence]
    test_sents_labels.append(sentence)

In [23]:
evaluator = Evaluator(y_te, y_pred, tags= [''], loader='list')

In [24]:
results, results_per_tag = evaluator.evaluate()

In [25]:
results

{'ent_type': {'correct': 462,
  'incorrect': 0,
  'partial': 0,
  'missed': 213,
  'spurious': 39,
  'possible': 675,
  'actual': 501,
  'precision': 0.9221556886227545,
  'recall': 0.6844444444444444,
  'f1': 0.7857142857142857},
 'partial': {'correct': 432,
  'incorrect': 0,
  'partial': 30,
  'missed': 213,
  'spurious': 39,
  'possible': 675,
  'actual': 501,
  'precision': 0.8922155688622755,
  'recall': 0.6622222222222223,
  'f1': 0.7602040816326531},
 'strict': {'correct': 432,
  'incorrect': 30,
  'partial': 0,
  'missed': 213,
  'spurious': 39,
  'possible': 675,
  'actual': 501,
  'precision': 0.8622754491017964,
  'recall': 0.64,
  'f1': 0.7346938775510203},
 'exact': {'correct': 432,
  'incorrect': 30,
  'partial': 0,
  'missed': 213,
  'spurious': 39,
  'possible': 675,
  'actual': 501,
  'precision': 0.8622754491017964,
  'recall': 0.64,
  'f1': 0.7346938775510203}}